In [31]:
import json

gxd_data = '../sample_data/gxd.json'

code2modality = {
  0: "histogram",
  1: "line chart",
  2: "other diagram",
  3: "macromolecule sequence",
  4: "3D structure",
  5: "fluorescence",
  6: "gel/blot",
  7: "plate",
  8: "light microscopy",
  9: "other",
  10: "residual",
  11: "compound"
}

with open(gxd_data, 'r') as f:
  data = json.load(f)

In [32]:
def parseEntry(entry: dict) -> dict:
  modalities = []
  for figure in entry['figures']:
    for subfigure in figure['subfigures']:
      modalities.append(code2modality[subfigure['type']])

  return {
    "cord_uid": entry['jaxid'],
    "source_x": 'gxd',
    "title": entry['title'],
    "abstract": entry['abstract'],
    "publish_time": f"{entry['year']}-01-01",
    "journal": "",
    "authors": "",
    "url": "",
    "pmcid": "",
    "modalities": ";".join(modalities),
  }


In [33]:
parsed_entries = []
for entry in data:
  parsed_entries.append(parseEntry(entry))

In [34]:
import pandas as pd 
df = pd.DataFrame(parsed_entries)

In [35]:
df.head()

,cord_uid,source_x,title,abstract,publish_time,journal,authors,url,pmcid,modalities
0,5792639,gxd,MicroRNA-127 Promotes Mesendoderm Differentiat...,Specification of the three germ layers is a fu...,2016-01-01,,,,,line chart;line chart;line chart;line chart;li...
1,5754472,gxd,The number of X chromosomes influences protect...,AIM: Sex differences in coronary heart disease...,2014-01-01,,,,,compound;line chart;line chart;line chart;line...
2,5430698,gxd,Regulated Expression of Chromobox Homolog 5 Re...,"The gene-trap lacZ reporter insertion, ROSA11,...",2012-01-01,,,,,line chart;histogram;other diagram;fluorescenc...
3,5758794,gxd,White spotting phenotype induced by targeted R...,Neural crest cells (NCCs) emerge from the dors...,2015-01-01,,,,,compound;residual;residual;residual;residual;r...
4,5553023,gxd,Transcriptional activation of hypoxia-inducibl...,Emerging evidence indicates that myeloid cells...,2014-01-01,,,,,residual;line chart;line chart;other diagram;o...


In [36]:
output_path = "../sample_data/gxd.parquet"
df.to_parquet(output_path, index=None)